This notebook shows an example recommendation system using doc2vec. We will use a dataset called CMU Book summaries [dataset](http://www.cs.cmu.edu/~dbamman/booksummaries.html). Alternateively, the dataset's link can be found in the `BookSummaries_Link.md` file under the Data folder in Ch7.


In [1]:
!pip install gensim
!pip install nltk
# todo : ask space engineer
#todo: add pip for downloading nltk data?

  Using cached numpy-1.20.2-cp39-cp39-win_amd64.whl (13.7 MB)
  Running setup.py clean for gensim
Failed to build gensim
    Running setup.py install for gensim: started
    Running setup.py install for gensim: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\sukee\appdata\local\programs\python\python39\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\sukee\\AppData\\Local\\Temp\\pip-install-evz5l5s9\\gensim\\setup.py'"'"'; __file__='"'"'C:\\Users\\sukee\\AppData\\Local\\Temp\\pip-install-evz5l5s9\\gensim\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\sukee\AppData\Local\Temp\pip-wheel-ifg133lk'
       cwd: C:\Users\sukee\AppData\Local\Temp\pip-install-evz5l5s9\gensim\
  Complete output (381 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\gensim
  copying gensim\downloader.py -> build\lib.win-amd64-3.9\gensim
  copying gensim\interfaces.py -> build\lib.win-amd64-3.9\gensim
  copying g

You should consider upgrading via the 'c:\users\sukee\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [3]:
#Read the dataset’s README to understand the data format. 
data_path = "booksummaries.txt"
mydata = {} #titles-summaries dictionary object
for line in open(data_path, encoding="utf-8"):
    temp = line.split("\t")
    mydata[temp[2]] = temp[6]



In [4]:
#prepare the data for doc2vec, build and save a doc2vec model
train_doc2vec = [TaggedDocument((word_tokenize(mydata[t])), tags=[t]) for t in mydata.keys()]
model = Doc2Vec(vector_size=50, alpha=0.025, min_count=10, dm =1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")



In [5]:
#Use the model to look for similar texts
model= Doc2Vec.load("d2v.model")

#This is a sentence from the summary of “Animal Farm” on Wikipedia:
#https://en.wikipedia.org/wiki/Animal_Farm
sample = """
Napoleon enacts changes to the governance structure of the farm, replacing meetings with a committee of pigs who will run the farm.
 """
new_vector = model.infer_vector(word_tokenize(sample))
sims = model.docvecs.most_similar([new_vector]) #gives 10 most similar titles
print(sims)

[('Animal Farm', 0.6777619123458862), ('The Wild Irish Girl', 0.6119967699050903), ("Snowball's Chance", 0.60667884349823), ('Family Matters', 0.5831906199455261), ('Settlers in Canada', 0.582908570766449), ('Poor White', 0.5771366953849792), ('The Road to Omaha', 0.576944887638092), ('Ponni', 0.5766265988349915), ("Family Guy: Stewie's Guide to World Domination", 0.5674009323120117), ('Texas Fever', 0.5643234848976135)]
